# Elastic Container Service

This notebook explores the AWS Elastic Container Service.

## Set Up

For this notebook, we will want to create a VPC with public and private subnets for our ECS cluster.
To keep the focus on ECS, we'll use cloud formation to create the VPC from the vpc template
in the [ecs-sample](https://github.com/d-smith/ecs-sample) project.

This notebook will use the [AWS Python SDK](https://aws.amazon.com/sdk-for-python/)

In [ ]:
import boto3

cf = boto3.client('cloudformation')

In [ ]:
# Here set the vpc_template to the s3 URL of your network template. Also change the stack name
# if you have already instantiated a stack with this name
vpc_template = 'https://s3-us-west-1.amazonaws.com/xtdc-deploy-templates/vpc-pub-priv.yaml'
stack_name = "network-us-west-2"

In [ ]:
response = cf.create_stack(
    StackName=stack_name,
    TemplateURL=vpc_template
)

print response

In [ ]:
waiter = cf.get_waiter('stack_create_complete')
waiter.wait(
    StackName=stack_name
)

print 'The wait is over for ', stack_name

## ECS Cluster

Once our VPC is in place, we'll need a cluster of machines to run our services and tasks on. For the
cluster, we will bring the following together:

* ECS Cluster Definition
* Role definition for cluster instance security context
* Launch configuration for launching machines to join the cluster
* Autoscaling policy for growing and shrinking the cluster.

In [ ]:
# Instantiate the ECS client
ecs = boto3.client('ecs')

In [ ]:
# Create a cluster
response = ecs.create_cluster(
    clusterName='myCluster'
)
print response

## Clean Up

In [ ]:
# Destroy cluster
response = ecs.delete_cluster(
    cluster='myCluster'
)

print response

In [ ]:
# Tear down network stack
response = cf.delete_stack(
    StackName=stack_name
)

print response

waiter = cf.get_waiter('stack_delete_complete')
print 'Waiting for delete for VPC stack...'
waiter.wait()
print 'Stack destroyed'